# Section 0 - Useful functions for your project

In [1]:
# Import useful libraries
import numpy as np
import scipy.io as io
import scipy.optimize as opt
import os

from tqdm import tqdm
from itertools import combinations


SCRIPT_DIR = os.getcwd()
X_STANDARD_SOL_PATH = os.path.join(SCRIPT_DIR, 'x_standard.npy')

Here-under we define the functions to:
- Encode a sentence into a binary vector
- Decode a binary vector into a sentence

In [2]:
def encoding_bin(mess):
    # Convert each character to its ASCII value and then to binary
    xi = [format(ord(char), '08b') for char in mess]

    # Get the number of characters
    m = len(xi)

    # Initialize an empty list for the binary vector
    x = []

    # Convert each binary string to a binary vector
    for i in range(m):
        x.append([int(bit) for bit in xi[i]])

    # Convert the list to a numpy array for easier manipulation
    x = np.array(x)


    # Return the binary vector and its dimensions
    d = x.shape[1]  # Number of bits per character
    x = x.flatten() # convert into a 1-d vector
    return x, d

def decoding_bin(x, d):
    # Ensure x is a binary vector (0s and 1s)
    x = np.clip(x, 0, 1)  # Clip values to be between 0 and 1
    x = np.round(x)        # Round values to the nearest integer

    # Initialize the output array
    y = np.zeros((len(x) // d, d), dtype=int)

    k = 0
    for i in range(len(x) // d):
        for j in range(d):
            y[i, j] = int(x[k])  # Fill the binary matrix
            k += 1

    # Convert binary to decimal and then to characters
    mess = ''.join(chr(int(''.join(map(str, row)), 2)) for row in y)

    return mess, y

In [3]:
message_in = "So happy to see you"
print("Message sent:", message_in)
binary_vector, dimensions = encoding_bin(message_in)
print("Binary Vector:\n", binary_vector)
float_vector = binary_vector.astype(np.float32)
message_decoded, binary_matrix = decoding_bin(float_vector, dimensions)
print("Decoded message:", message_decoded)

Message sent: So happy to see you
Binary Vector:
 [0 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0
 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1
 1 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 1 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 1
 0 1 0 1]
Decoded message: So happy to see you


In [4]:
dimensions

8

The function below simulates the effect of the noisy channel

In [5]:
# Disrupts percenterror% of y entries randomly
def noisychannel(y, percenterror):
    m = len(y)                               # Length of the message
    K = int(np.floor(m * percenterror))      # Number of entries to corrupt
    I = np.random.permutation(m)[:K]         # Random indices to corrupt
    y_n = np.copy(y)                         # Copy of the orginal message
    vec = np.random.rand(K) * np.mean(y)
    y_n[I] = vec                             # Corruption of selected inputs
    return y_n

In [6]:
# Try it
message_in = "A crystal clear message"
binary_vector, dimensions = encoding_bin(message_in)
percenterror = 0.05
float_vector = binary_vector.astype(np.float32)
yprime = noisychannel(float_vector, percenterror)
print("Message sent:", message_in)
message_corr_decoded, binary_matrix = decoding_bin(yprime, dimensions)
print("Decoded noisy message:", message_corr_decoded)

Message sent: A crystal clear message
Decoded noisy message: A crystal cDeAr messa'e


In [7]:
float_vector.shape

(184,)

# Section 1 - Decode the Message from Alice

In [8]:
# Load your mat file in Python
## Once your team is built, contact the Instructors by email to mention who is part of the group.
## You will then receive by return email your personal message from Alice to decrypt in .mat file.
## Alert: do not share it !

data = io.loadmat('messageFromAlice.mat')
# data is dictionnay where
## data['A'] is the encoding matrix exchanged between Alice and Bob
## data['d'] is the dimension
## data['yprime'] is the encrypted message received from Alice

# Load the arrays
A = data['A']
yprime = data['yprime'].T
yprime = np.squeeze(yprime)
d = data['d'][0][0]

In [9]:
A.shape, yprime.shape

((3424, 856), (3424,))

In [10]:
# let's create some utility functions
def get_standard_format_matrix(A: np.ndarray) -> np.ndarray:
    m, p = A.shape
    I = np.eye(m)
    # let's prepare the 4 blocks
    ab1 = np.concatenate([-A, I, -np.eye(m), np.zeros((m, m)), np.zeros((m, p))], axis=1)
    ab2 = np.concatenate([A, I, np.zeros((m, m)), -np.eye(m), np.zeros((m, p))], axis=1)
    ab3 = np.concatenate([-np.eye(p), np.zeros((p, m)), np.zeros((p, m)), np.zeros((p, m)), -np.eye(p)], axis=1)
    
    Astandard = np.concatenate([ab1, ab2, ab3], axis=0)

    assert Astandard.shape == (2 * m + p, 2 * p + 3 * m), f"Expected {(2 * m + p, 2 * p + 3 * m)}. Found: {Astandard.shape}"

    return Astandard

def get_cost_coefficients(A: np.ndarray) -> np.ndarray:
    m, p = A.shape
    # let's build the cost coefficient 
    c = np.concatenate([np.zeros((1, p)), 
                        np.ones((1, m)),
                        np.zeros((1, m)),
                        np.zeros((1, m)),
                        np.zeros((1, p)),                        
                        ], 
                        axis=1).squeeze() 

    # c = [0, 0, 0, 0, 0...1, 1, 1,] (p 0s and m 1s)
    assert c.shape == (2 * p + 3 * m, ), f"Expected {(2 * p + 3 * m, )}. Found: {c.shape}"

    return c

def get_eq_constraints(A: np.ndarray, y_noise: np.ndarray) -> np.ndarray:
    m, p = A.shape
    y_2d = np.expand_dims(y_noise, axis=0)
    b_standard = np.concatenate([-y_2d, y_2d, -np.ones((1, p))], axis=1).squeeze()
    assert b_standard .shape == (2 * m + p,), f"Expected {(2 * m + p,)}. Found: {b_standard .shape}"
    return b_standard


In [11]:
def decode_secrete_message(A: np.ndarray, y_noise: np.ndarray):
    # according to the scipy.linprog documentation
    # the function accepts 5 arguments
    # c, Aub, bub, Aeq, b_eq, lb, ub

    # minimize: c @ x
    # such that
    # A_ub @ x <= b_ub
    # A_eq @ x == b_eq
    # lb <= x <= ub

    c = get_cost_coefficients(A)
    Astandard = get_standard_format_matrix(A)
    b_standard = get_eq_constraints(A, y_noise)

    x_standard = opt.linprog(c, A_ub=None, b_ub=None, A_eq=Astandard, b_eq=b_standard, bounds=(0, None), method='revised simplex').x
    return x_standard

Use your algorithm to solve

$min_{0 <= x^{'} <= 1} ||A*x^{'} - y^{'}||_1$


In [12]:
# save x_standard once instead of rerunning everytime
if not os.path.exists(X_STANDARD_SOL_PATH):
    x_standard = decode_secrete_message(A, yprime)
    np.save(X_STANDARD_SOL_PATH, x_standard)
else:
    x_standard = np.load(X_STANDARD_SOL_PATH)

# since x_standard contains xprime, t and slack variables, we need to extract x' before proceeding
xprime = x_standard[:A.shape[1]]

In [13]:
# Display the result:
d = 8    # Number of bits per character
message_decoded, binary_matrix = decoding_bin(xprime, d)
print("The recovered message is:", message_decoded)

The recovered message is: You can claim your personal reward by going to Student affairs, giving you code=1350 and ask for you reward


In [14]:

# def is_invertible(matrix: np.ndarray):
#     # determining whether a matrix numerically is tricky...
#     # using the following approach:
#     # https://stackoverflow.com/questions/17931613/how-to-decide-a-whether-a-matrix-is-singular-in-python-numpy

#     if matrix.shape[0] != matrix.shape[1]:
#         raise ValueError(f"Make sure to pass an invertible matrix. found matrix with shape: {matrix.shape}")
    
#     return np.linalg.matrix_rank(matrix) == matrix.shape[0]

In [15]:
# def isVertex(A: np.ndarray, solution: np.ndarray, zero_thresh: float = 10 ** -12) -> bool:
#     Astadard = get_standard_format_matrix(A)

#     m, n = Astadard.shape

#     # consider values less than a certain threshold as zero
#     c_sol = np.clip(solution, zero_thresh, 1)
#     c_sol = c_sol  * (c_sol >= zero_thresh).astype(float) # any value less than 'zero_thresh' will be set to 0

#     zero_entries_indices = [i for i, v in enumerate(solution) if v == 0]

#     print(f"found {len(zero_entries_indices)} zero entries with n - m equals: {n - m}")

#     # find all the combinations of set with (n - m) zero entries
#     cs = list(combinations(zero_entries_indices, n - m))

#     for null_entries in tqdm(cs, desc="iterating through possible combinations"):
#         # convert the null_entries to a set
#         null_entries = set(null_entries)
#         # extract the basic_entries
#         basic_entries = [i for i, _ in enumerate(solution) if i not in null_entries]

#         # extract the submatrix
#         basic_submatrix = Astadard[:, basic_entries]
#         # make sure the shape is correct
#         assert basic_submatrix.shape == (m, m)

#         # check the invertibility of the basic submatrix 
#         is_basic = is_invertible(basic_submatrix)

#         # if we found an invertible, then our job is done here
#         if is_basic:    
#             return True
        
#         # otherwise... move to the next candidate set of null entries.

#     return False

In [16]:
# isVertex(A = A, solution=x_standard)

# Section 2 - Generate and Decode your own messages

This section is dedicated to the fifth question of the project:
- Sending an encrypted message through a channel with sparse Gaussian noise...
- Encode and decode a message yourself:

In [17]:
#  Message to send
my_mess = "Hey ! Welcome "

# Message in binary form
binary_vector, d = encoding_bin(my_mess)
x = binary_vector.astype(np.float32)

# Length of the message
size = x.shape
n = size[0]

# Length of the message which will be sent
m = 4*n

# Encoding matrix: we take a randomly generated matrix
A = np.random.randn(m,n)

# Message you wish to send
y = A@x

# Noise added by the transmission channel
# = normal N(0,sigma) for a % input of y
percenterror = 0.05
yprime = noisychannel(y, percenterror)

Find x approximately from yprime by solving:


$min_{0 <= x^{'} <= 1} ||A*x^{'} - y^{'}||_1$

In [18]:
# xprime = yourAlgorithm(A, yprime)

In [20]:
from dankin_method import dankin_algorithm
cost_vec = get_cost_coefficients(A)
b_eq = get_eq_constraints(A, yprime)
A_eq = get_standard_format_matrix(A)

In [22]:
x_dankin = dankin_algorithm(A_eq, b_eq, cost_vec, 10 ** -3)

KeyboardInterrupt: 

In [19]:
# Display the result:
d = 8    # Number of bits per character
message_decoded, binary_matrix = decoding_bin(xprime, d)
print("The recovered message is:", message_decoded)
print("The error is:", np.linalg.norm(x-xprime))

The recovered message is: You can claim your personal reward by going to Student affairs, giving you code=1350 and ask for you reward


ValueError: operands could not be broadcast together with shapes (112,) (856,) 

# Section 3 - Dikin's Method

This section is dedicated to the sixth question of the project:
- Implement the Dikin's Method and compare its results with the previous ones.

In [1]:
from dankin_method import dankin_algorithm

#  Message to send
my_mess = "Hey ! Welcome "

# Message in binary form
binary_vector, d = encoding_bin(my_mess)
x = binary_vector.astype(np.float32)

# Length of the message
size = x.shape
n = size[0]

# Length of the message which will be sent
m = 4*n

# Encoding matrix: we take a randomly generated matrix
A = np.random.randn(m,n)

# Message you wish to send
y = A@x

# Noise added by the transmission channel
# = normal N(0,sigma) for a % input of y
percenterror = 0.05
yprime = noisychannel(y, percenterror)




ImportError: cannot import name 'dankin_algorithm' from 'dankin_method' (/home/ayhem18/DEV/optimization_project/dankin_method.py)

# Section 4 - Integer Programming

This section is dedicated to the seventh question of the project:
- by imposing binary variables: can you recover your message with a higher noise level?